In [1]:
import pandas as pd
import numpy as np
import os
import math

import librosa

from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
import matplotlib.pyplot as plt

In [2]:
# path_to_file = 'C:/Users/USER/Downloads/Tugas Akhir/dataset/tes/'
# path_to_fitur = "C:/Users/USER/Downloads/Tugas Akhir/dataset/tes/fitur/"
# path_to_model = "C:/Users/USER/Downloads/Tugas Akhir/dataset/model/"

path_to_file = 'F:/178/Tugas Akhir/dataset/tes/'
path_to_fitur = "F:/178/Tugas Akhir/dataset/tes/fitur/"
path_to_model = "F:/178/Tugas Akhir/dataset/model/"

In [6]:
# #Load
def load(src):
    y, sr = librosa.load(src,mono=True,sr=22050)
    
    return y,sr

# #Convert
def convert(filename,path,y,sr):
    #Path
    dst = path+filename[:-4]+".wav"
    
    #COnver to WAV
    librosa.output.write_wav(dst, y, sr)

# #Get Feature

# #MFCC
def mfcc(y,sr,file):
    vector = list()
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13) #13 is default dimension 512 frame
    data = pd.DataFrame(mfcc)
    mean = data.mean(axis = 1).values #Get Mean
    std = data.std(axis = 1).values #Get std
    vector.append(np.insert(mean,13,std))
    save = pd.DataFrame(vector)

    save.to_csv(path_to_fitur+file[:-4]+"/mfcc.csv", index=False, header=False)

# #TIMBRE // CENTROID // FLUX // ROLLOFF // ZERO CROSSING 
def timbre(y,sr,file):
    vector = list()
    cent = librosa.feature.spectral_centroid(y=y, sr=sr) #1 is default dimension 512 frame
    flux = librosa.onset.onset_strength(y=y, sr=sr) #1 is default dimension 512 frame
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr) #1 is default dimension 512 frame
    zc = librosa.feature.zero_crossing_rate(y) #1 is default dimension 512 frame
    
    a = pd.DataFrame(cent) 
    b = pd.DataFrame(flux).T
    c = pd.DataFrame(rolloff)
    d = pd.DataFrame(zc)
    
    frame = [a,b,c,d]
    data = pd.concat(frame)
    mean = data.mean(axis = 1).values #Get Mean
    std = data.std(axis = 1).values #Get std
    vector.append(np.insert(mean,4,std))
    save = pd.DataFrame(vector)
    
    save.to_csv(path_to_fitur+file[:-4]+"/timbre.csv", index=False, header=False)

# #SCF and SFM
def flatness(y,sr,file):
    vector = list()
    flatness = librosa.feature.spectral_flatness(y=y)
    data = pd.DataFrame(flatness)
    mean = data.mean(axis = 1).values #Get Mean
    std = data.std(axis = 1).values #Get std
    vector.append(np.insert(mean,1,std))
    save = pd.DataFrame(vector)
    
    save.to_csv(path_to_fitur+file[:-4]+"/flatness.csv", index=False, header=False)
    
def crest(y,sr,file):
    peak = y.max()
    rms = librosa.feature.rmse(y=y)
    n = rms.size
    square = rms**2
    rms_v2 = math.sqrt((1/n)*(square.sum()))
    crest = [peak/rms_v2]
    
    save = pd.DataFrame(crest)
    
    save.to_csv(path_to_fitur+file[:-4]+"/crest.csv", index=False, header=False)

# #chroma
def chroma(y,sr,file):
    vector = list()
    chroma = librosa.feature.chroma_stft(y=y, sr=sr, n_chroma=12) #24 is default dimension 512 frame
    data = pd.DataFrame(chroma)
    mean = data.mean(axis = 1).values #Get Mean
    std = data.std(axis = 1).values #Get std
    vector.append(np.insert(mean,12,std))
    save = pd.DataFrame(vector)
    
    save.to_csv(path_to_fitur+file[:-4]+"/chroma.csv", index=False, header=False)

def fitur(file,y,sr):
    
    mfccs = mfcc(y,sr,file) #mfcc
    timbres = timbre(y,sr,file) #timbre
    flatnesss = flatness(y,sr,file) #sfm
    crests = crest(y,sr,file) #scf
    chromas = chroma(y,sr,file) #chroma
    
# #merge feature

def merge(case1,case2):
    case_a = pd.DataFrame(case1)
    case_b = pd.DataFrame(case2)
    
    merged = case_a.merge(case_b, how='outer', left_index=True, right_index=True)
    
    return merged

def flat(song,fitur):
    vector = list()
    
    data = pd.read_csv(path_to_fitur+str(song)+"/"+fitur+".csv",header=None)
    vector.append(np.squeeze(data.values,0))

    return vector

def case1(file):
    fitur = 'mfcc'
    result = flat(file,fitur)
    df = pd.DataFrame(result)
    
    return df

def case2(file):
    case_1 = case1(file)
    fitur = 'timbre'
    case_2 = flat(file,fitur)
    result = merge(case_1,case_2)
    
    return result

def case3(file):
    case_2 = case2(file)
    fitur1 = 'crest'
    fitur2 = 'flatness'
    crest = flat(file,fitur1)
    flatness = flat(file,fitur2)
    results = merge(case_2,crest)
    result = merge(results,flatness)
    
    return result

def case4(file):
    case_3 = case3(file)
    fitur = 'chroma'
    chroma = flat(file,fitur)
    result = merge(case_3,chroma)
    
    return result

def norm(data):
    result = np.array(data)
    mean = result.mean()
    std = result.std()

    normalized = (result-mean)/std
    
    return normalized

# #Get emotion

def emotion(a,v):
    if ((v > 0) & (a >= 0)):
        cluster = 'cluster 1'
    if ((v <= 0) & (a > 0)):
        cluster = 'cluster 2'
    if ((v < 0) & (a <= 0)):
        cluster = 'cluster 3'
    if ((v >= 0) & (a < 0)):
        cluster = 'cluster 4'
        
    return cluster

def plot_scatter(a,v):
    plt.figure(figsize=(15, 15))
    plt.xlim(-1,1)
    plt.ylim(-1,1)
    bbox=dict(boxstyle="round",
              ec=(1., 0.5, 0.5),
              fc=(1., 0.8, 0.8),
              alpha=0.6,
              color = 'yellow'
           )
    plt.text(0.6, 0.7, "cluster 1", size=30,
         ha="center", va="center",
         bbox=bbox
         )
    plt.text(-0.6, 0.7, "cluster 2", size=30,
         ha="center", va="center",
         bbox=bbox
         )
    plt.text(-0.6, -0.7, "cluster 3", size=30,
         ha="center", va="center",
         bbox=bbox
         )
    plt.text(0.6, -0.7, "cluster 4", size=30,
         ha="center", va="center",
         bbox=bbox
         )

    plt.scatter(v,a,color='b')
    plt.axhline(y=0,color='black', label ="X")
    plt.axvline(x=0,color='black', label ="Y")
    plt.show()
    
def proses(path):
    #Split path
    drive, path_and_file = os.path.splitdrive(path)
    paths, file = os.path.split(path_and_file)
    path_wav = drive+paths+'\wav\\'
    path_fitur = drive+paths+'\fitur\\'
    #load raw
    y, sr = load(path)

    #Convert Wav
    convert(file,path_wav,y,sr)

    #Load Wav
    y_wav, sr_wav = load(path_wav+file[:-4]+'.wav')

    #make dir
    try:  
        os.mkdir(path_fitur+file[:-4])
    except OSError:  
        print ("Creation of the directory %s failed" % path_fitur)

    # #Feature extraction
    fiture = fitur(file[:-4]+'.wav',y_wav,sr_wav)
    print("feature successfully")

    # #Flatten
    results = case4(file[:-4])
    print("flatten successfully")

    # #Norm
    normalized = norm(results)
    
    # #Load Model

    arousal = path_to_model+'arousal/case4.sav'
    valence = path_to_model+'valence/case4.sav'

    arousal_model = joblib.load(arousal)
    valence_model = joblib.load(valence)

    a = arousal_model.predict(normalized)
    v = valence_model.predict(normalized)
    
    return v,a,normalized

#     emo = emotion(a,v)

In [7]:
v,a,normalized = proses("F:\\178\\Tugas Akhir\\dataset\\tes\\939.mp3")
print(v,a)

Creation of the directory F:\178\Tugas Akhir\dataset\tesitur\ failed
feature successfully
flatten successfully
[ 0.03546424] [ 0.06303378]


In [8]:
normalized

array([[-0.40123516, -0.17436114, -0.21201893, -0.22711447, -0.21631734,
        -0.23536267, -0.24334351, -0.23794073, -0.23502287, -0.2288384 ,
        -0.24186593, -0.23976093, -0.23539413, -0.12907272, -0.18490736,
        -0.20716278, -0.21445586, -0.21253925, -0.21694539, -0.21949578,
        -0.21852018, -0.22349165, -0.22279385, -0.22309316, -0.22342855,
        -0.22392223,  3.07838357, -0.23241923,  6.46844483, -0.23410529,
         1.35590237, -0.23238714,  2.18854562, -0.23412086, -0.2266885 ,
        -0.23422704, -0.2342127 , -0.23374873, -0.233826  , -0.23396537,
        -0.23398459, -0.23400574, -0.2340055 , -0.23399544, -0.23398187,
        -0.23394542, -0.23389349, -0.23384538, -0.23382258, -0.23394234,
        -0.23394722, -0.23397674, -0.23397432, -0.2339969 , -0.23398323,
        -0.23399808, -0.23399478, -0.23399258, -0.23397716, -0.23394951,
        -0.23395771]])

In [ ]:
# [-0.38665231][-0.75646618] [ 0.06427516][-0.03813184] [ 0.11686378][-0.00219273] [ 0.20947236][ 0.12625835]
# [-0.31986225][-0.6172328] [ 0.04008732][-0.06173782] [ 0.0858818][-0.0337648] [ 0.16975319][ 0.08035165]
# [-0.16978618] [-0.3717958]
#Load Model

# arousal = path_to_model+'arousal/case1.sav'
# valence = path_to_model+'valence/case2.sav'

# arousal_model = joblib.load(arousal)
# valence_model = joblib.load(valence)
# valence_model.get_params()
# a = arousal_model.predict(result)
# v = valence_model.predict(result)

# emo = emotion(a,v)

In [ ]:
# for filename in os.listdir(path_to_file):
#     if os.path.isfile(os.path.join(path_to_file, filename)):
        
#         #load raw
#         y, sr = load(path_to_file+filename)

#         #Convert Wav
#         convert(filename,path_dst,y,sr)
        
#         #Load Wav
#         y_wav, sr_wav = load(path_to_file+'wav/'+file)

#         #make dir
#         try:  
#             os.mkdir(path_to_fitur+filename[:-4])
#         except OSError:  
#             print ("Creation of the directory %s failed" % path_to_fitur)
        
#         # #Feature extraction
#         fiture = fitur(filename[:-4]+'.wav',y_wav,sr_wav)
#         print("feature successfully")
        
#         # #Flatten
#         results = case4(filename[:-4])
#         print("flatten successfully")
        
#         # #Norm
#         normalized = norm(results)